# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# import weather data
weather_data_df = pd.read_csv("../output_data/cities.csv")
weather_data_df = weather_data_df.drop(columns=weather_data_df.columns[0], axis=1)
weather_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bathsheba,13.2167,-59.5167,77.61,83,75,13.80,BB,1644191736
1,lompoc,34.6391,-120.4579,77.09,52,0,11.50,US,1644191737
2,yellowknife,62.4560,-114.3525,-1.68,84,75,3.44,CA,1644191517
3,rikitea,-23.1203,-134.9692,79.32,71,3,18.54,PF,1644191718
4,castro,-24.7911,-50.0119,65.37,83,47,1.86,BR,1644191737
...,...,...,...,...,...,...,...,...,...
564,muswellbrook,-32.2667,150.9000,69.24,61,34,12.57,AU,1644192634
565,juneau,58.3019,-134.4197,40.84,99,100,8.99,US,1644192559
566,marsh harbour,26.5412,-77.0636,74.08,73,76,12.35,BS,1644192635
567,te anau,-45.4167,167.7167,61.68,51,48,2.37,NZ,1644192635


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# dependencies
import gmaps
import gmaps.datasets
# gmaps 
locations = weather_data_df[["Lat","Lng"]].astype(float)
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()
gmaps.configure(api_key=g_key)
# add heat layer 
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius = 1.5) 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
filtered_weather_df = weather_data_df
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]
filtered_weather_df.dropna()
filtered_weather_df = filtered_weather_df.head(10)
filtered_weather_df
 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,port elizabeth,-33.9180,25.5701,72.10,73,0,4.61,ZA,1644191676
79,yulara,-25.2406,130.9889,76.84,36,0,8.05,AU,1644191820
112,isla vista,34.4133,-119.8610,79.27,31,0,1.99,US,1644191890
117,san lorenzo,-25.3333,-57.5333,79.74,44,0,6.91,PY,1644191891
128,alice springs,-23.7000,133.8833,76.55,47,0,0.00,AU,1644191815
132,codrington,-38.2667,141.9667,79.03,32,0,5.57,AU,1644191896
140,saint-pierre,-21.3393,55.4781,76.68,78,0,5.75,RE,1644191958
143,pacific grove,36.6177,-121.9166,70.45,66,0,4.00,US,1644191959
270,warri,5.5167,5.7500,75.92,85,0,3.29,NG,1644192122
331,tchaourou,8.8865,2.5975,77.43,19,0,2.62,BJ,1644192262


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = filtered_weather_df

In [21]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Not found...skipping.")
        
    print("------------")

Retrieving Results for Index 78: port elizabeth.
Closest hotel is 39 On Nile Guest House.
------------
Retrieving Results for Index 79: yulara.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 112: isla vista.
Closest hotel is Best Western Plus South Coast Inn.
------------
Retrieving Results for Index 117: san lorenzo.
Closest hotel is Hotel del Sur.
------------
Retrieving Results for Index 128: alice springs.
Closest hotel is Desert Palms Alice Springs.
------------
Retrieving Results for Index 132: codrington.
Closest hotel is Codrington Gardens Bed And Breakfast.
------------
Retrieving Results for Index 140: saint-pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for Index 143: pacific grove.
Closest hotel is Lovers Point Inn.
------------
Retrieving Results for Index 270: warri.
Closest hotel is BON Hotel Delta.
------------
Retrieving Results for Index 331: tchaourou.
Closest hotel is SAFIR HOTEL.
-------

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map, display figure
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))